<a href="https://colab.research.google.com/github/GAC2099/Chat2DB/blob/main/magic_animate_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q diffusers==0.21.4 transformers==4.32.0 accelerate==0.22.0 omegaconf==2.3.0 einops==0.6.1 av gradio
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

!apt -y install -qq aria2
from huggingface_hub import snapshot_download
snapshot_download(repo_id="zcxu-eric/MagicAnimate", local_dir="/content/MagicAnimate")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/139 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

densepose_controlnet/config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

appearance_encoder/config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

temporal_attention.ckpt:   0%|          | 0.00/5.11G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.43G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

'/content/MagicAnimate'

In [2]:
%cd /content
!git clone https://github.com/nicehero/magic-animate-for-colab
%cd /content/magic-animate-for-colab
!git submodule update --init --recursive
%cd /content
!git clone https://huggingface.co/bdsqlsz/control_v11p_sd15_openpose
BaseModelUrl = "https://huggingface.co/runwayml/stable-diffusion-v1-5"
BaseModelDir = "/content/magic-animate-for-colab/stable-diffusion-v1-5"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/model_index.json -d {BaseModelDir} -o model_index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/vae/diffusion_pytorch_model.bin -d {BaseModelDir}/vae -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/vae/config.json -d {BaseModelDir}/vae -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/unet/diffusion_pytorch_model.bin -d {BaseModelDir}/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/unet/config.json -d {BaseModelDir}/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/vocab.json -d {BaseModelDir}/tokenizer -o vocab.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/tokenizer_config.json -d {BaseModelDir}/tokenizer -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/special_tokens_map.json -d {BaseModelDir}/tokenizer -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/merges.txt -d {BaseModelDir}/tokenizer -o merges.txt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/text_encoder/pytorch_model.bin -d {BaseModelDir}/text_encoder -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/text_encoder/config.json -d {BaseModelDir}/text_encoder -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/scheduler/scheduler_config.json -d {BaseModelDir}/scheduler -o scheduler_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/safety_checker/pytorch_model.bin -d {BaseModelDir}/safety_checker -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/safety_checker/config.json -d {BaseModelDir}/safety_checker -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/feature_extractor/preprocessor_config.json -d {BaseModelDir}/feature_extractor -o preprocessor_config.json

/content
Cloning into 'magic-animate-for-colab'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 280 (delta 81), reused 55 (delta 55), pack-reused 185
Receiving objects: 100% (280/280), 24.99 MiB | 23.72 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/content/magic-animate-for-colab
Submodule 'video_controlnet_aux' (https://github.com/sdbds/video_controlnet_aux) registered for path 'video_controlnet_aux'
Cloning into '/content/magic-animate-for-colab/video_controlnet_aux'...
Submodule path 'video_controlnet_aux': checked out '9a86fb8077e0fae573ad5240d615bb0a08765874'
/content
Cloning into 'control_v11p_sd15_openpose'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (8/8), 1.41 KiB | 723.00 KiB/s, done.

Download R

In [3]:
#download https://civitai.com/models/43331/majicmix-realistic
#!mkdir /content/magic-animate-for-colab/pretrained_models/
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/176425?type=Model&format=SafeTensor&size=pruned&fp=fp16
#!mv /content/majicmixRealistic_v7.safetensors /content/magic-animate-for-colab/pretrained_models/


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
756504|OK  |   233MiB/s|/content/majicmixRealistic_v7.safetensors

Status Legend:
(OK):download completed.


In [4]:
#edit /content/magic-animate-for-colab/configs/prompts/animation.yaml
!cat /content/magic-animate-for-colab/configs/prompts/animation.yaml

pretrained_model_path: "/content/majicmixRealistic_v7.safetensors"
pretrained_vae_path: ""
pretrained_controlnet_path: "/content/MagicAnimate/densepose_controlnet"
openpose_path: "/content/control_v11p_sd15_openpose"
pretrained_appearance_encoder_path: "/content/MagicAnimate/appearance_encoder"
pretrained_unet_path: ""

motion_module: "/content/MagicAnimate/temporal_attention/temporal_attention.ckpt"

savename: null

fusion_blocks: "midup"

seed:           [1]
steps:          25
guidance_scale: 7.5

source_image:
  - "inputs/applications/source_image/monalisa.png"
  - "inputs/applications/source_image/0002.png"
  - "inputs/applications/source_image/demo4.png"
  - "inputs/applications/source_image/dalle2.jpeg"
  - "inputs/applications/source_image/dalle8.jpeg"
  - "inputs/applications/source_image/multi1_source.png"
video_path:
  - "inputs/applications/driving/densepose/running.mp4"
  - "inputs/applications/driving/densepose/demo4.mp4"
  - "inputs/applications/driving/densepose/demo4.mp

In [ ]:
%cd /content/magic-animate-for-colab
!python -m demo.gradio_animate

/content/magic-animate-for-colab
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2023-12-10 15:39:18.780817: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 15:39:18.780868: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 15:39:18.780907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 15:39:20.189955: W tensorflow/compiler/tf2tensorrt/utils/py_utils.c